In [1]:
# 將所有 csv 讀入同一個 df 入面
import pandas as pd
import glob

all_files = glob.glob("./*.csv")
file_list = []
for file in all_files:
    df = pd.read_csv(file, header=None, names=["label", "word", "jyutping"], index_col=False)
    file_list.append(df)

data = pd.concat(file_list, axis=0, ignore_index=True)

del df, file, all_files, file_list

讀晒所有數據之後，按照下面嘅規則提取有用嘅字詞出嚟
1. 所有標咗 1* 嘅詞條單獨列出嚟保留
1. 所有有粵拼嘅詞條都要保留
1. 剩低嘅詞條入面，凡係冇 1 嘅都踢走，有 1 嘅都留低

所以總共得到 3 個列表，然後分別去重排序

In [2]:
# 需要人手屋修改嘅詞彙
error_words = data[data["label"] == "1*"]

remain = data[data["label"] != "1*"]

# 有粵拼，全部都保留嘅詞彙
has_jyutping = remain[remain["jyutping"].notna()]
no_jyutping = remain[remain["jyutping"].isna()]


# 冇粵拼，但係人工揀出嚟有用嘅詞彙
jyutping_useful = has_jyutping[has_jyutping["label"].notna()]
jyutping_junk = has_jyutping[has_jyutping["label"].isna()]
no_jyutping_useful = no_jyutping[no_jyutping["label"].notna()]
no_jyutping_junk = no_jyutping[no_jyutping["label"].isna()]

del remain, has_jyutping, no_jyutping

error_words.sort_values(by=["jyutping"], inplace=True)
error_words.to_csv("output/error_words.csv", index=False)

jyutping_useful.sort_values(by=["jyutping"], inplace=True)
jyutping_useful.to_csv("output/jyutping_useful.csv", index=False)

jyutping_junk.sort_values(by=["jyutping"], inplace=True)
jyutping_junk.to_csv("output/jyutping_junk.csv", index=False)

no_jyutping_useful.sort_values(by=["jyutping"], inplace=True)
no_jyutping_useful.to_csv("output/no_jyutping_useful.csv", index=False)

no_jyutping_junk.sort_values(by=["jyutping"], inplace=True)
no_jyutping_junk.to_csv("output/no_jyutping_junk.csv", index=False)

/var/folders/7j/k3r8hd7n72v74g6tqfr9rl0m00xnff/T/ipykernel_37428/3868571460.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_words.sort_values(by=["jyutping"], inplace=True)
